In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("structured_streaming") \
    .getOrCreate()

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
# Import Data
aisles = spark.read.csv("/FileStore/tables/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("/FileStore/tables/departments.csv", header=True, inferSchema=True)
#order_products_prior = spark.read.csv("/FileStore/tables/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/FileStore/tables/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/FileStore/tables/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/FileStore/tables/products.csv", header=True, inferSchema=True)


In [4]:
%python
orders.createOrReplaceTempView("orders")
orders.show(10)

+--------+-------+--------+------------+---------+-----------------+----------------------+
order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
 2539329| 1| prior| 1| 2| 8| null|
 2398795| 1| prior| 2| 3| 7| 15.0|
 473747| 1| prior| 3| 3| 12| 21.0|
 2254736| 1| prior| 4| 4| 7| 29.0|
 431534| 1| prior| 5| 4| 15| 28.0|
 3367565| 1| prior| 6| 2| 7| 19.0|
 550135| 1| prior| 7| 1| 9| 20.0|
 3108588| 1| prior| 8| 1| 14| 14.0|
 2295261| 1| prior| 9| 1| 16| 0.0|
 2550362| 1| prior| 10| 4| 8| 30.0|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 10 rows

In [5]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from orders  
 group by order_dow 
 order by total_orders desc

total_orders,day_of_week
600905,Sunday
587478,Monday
467260,Tuesday
453368,Friday
448761,Saturday
436972,Wednesday
426339,Thursday


In [6]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from orders 
 group by order_hour_of_day 
 order by order_hour_of_day

total_orders,hour
22758,0
12398,1
7539,2
5474,3
5527,4
9569,5
30529,6
91868,7
178201,8
257812,9


In [7]:
%python
products.createOrReplaceTempView("products")
orders.createOrReplaceTempView("orders")
departments.createOrReplaceTempView("departments")

In [8]:
%sql
select d.department, count(distinct p.product_id) as products
  from products p
    inner join departments d
      on d.department_id = p.department_id
 group by d.department
 order by products desc
 limit 10

department,products
personal care,6563
snacks,6264
pantry,5371
beverages,4365
frozen,4007
dairy eggs,3449
household,3084
canned goods,2092
dry goods pasta,1858
produce,1684


In [9]:
%python
products.createOrReplaceTempView("products")
orders.createOrReplaceTempView("orders")
departments.createOrReplaceTempView("departments")
aisles.createOrReplaceTempView("aisles")
order_products_train.createOrReplaceTempView("order_products_train")

In [10]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [11]:
rawData.show(10,truncate = False)

+---------------------------------------------+--------+
product_name |order_id|
+---------------------------------------------+--------+
Bulgarian Yogurt |1 |
Organic 4% Milk Fat Whole Milk Cottage Cheese|1 |
Organic Celery Hearts |1 |
Cucumber Kirby |1 |
Lightly Smoked Sardines in Olive Oil |1 |
Bag of Organic Bananas |1 |
Organic Hass Avocado |1 |
Organic Whole String Cheese |1 |
Grated Pecorino Romano Cheese |36 |
Spring Water |36 |
+---------------------------------------------+--------+
only showing top 10 rows

In [12]:
baskets.show(20)

+--------+--------------------+
order_id| items|
+--------+--------------------+
 1342|[Raw Shrimp, Seed...|
 1591|[Cracked Wheat, S...|
 4519|[Beet Apple Carro...|
 4935| [Vodka]|
 6357|[Globe Eggplant, ...|
 10362|[Organic Baby Spi...|
 19204|[Reduced Fat Crac...|
 29601|[Organic Red Onio...|
 31035|[Organic Cripps P...|
 40011|[Organic Baby Spi...|
 46266|[Uncured Beef Hot...|
 51607|[Donut House Choc...|
 58797|[Concentrated But...|
 61793|[Raspberries, Gre...|
 67089|[Original Tofurky...|
 70863|[Extra Hold Non-A...|
 88674|[Organic Coconut ...|
 91937|[No. 485 Gin, Mon...|
 92317|[Red Vine Tomato,...|
 99621|[Organic Baby Aru...|
+--------+--------------------+
only showing top 20 rows

In [13]:
%python
df1 = baskets.join(rawData, ["order_id"])
df1.show(10)

+--------+--------------------+--------------------+
order_id| items| product_name|
+--------+--------------------+--------------------+
 1342|[Raw Shrimp, Seed...|Bag of Organic Ba...|
 1342|[Raw Shrimp, Seed...| Seedless Cucumbers|
 1342|[Raw Shrimp, Seed...| Organic Mandarins|
 1342|[Raw Shrimp, Seed...|Organic Strawberries|
 1342|[Raw Shrimp, Seed...|Versatile Stain R...|
 1342|[Raw Shrimp, Seed...| Pink Lady Apples|
 1342|[Raw Shrimp, Seed...|Chicken Apple Sau...|
 1342|[Raw Shrimp, Seed...| Raw Shrimp|
 1591|[Cracked Wheat, S...| Cracked Wheat|
 1591|[Cracked Wheat, S...|Organic Greek Who...|
+--------+--------------------+--------------------+
only showing top 10 rows

In [14]:
df = df1["items","order_id"]
df.show(10)

+--------------------+--------+
 items|order_id|
+--------------------+--------+
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Raw Shrimp, Seed...| 1342|
[Cracked Wheat, S...| 1591|
[Cracked Wheat, S...| 1591|
+--------------------+--------+
only showing top 10 rows

In [15]:
from pyspark.ml.fpm import FPGrowth


In [16]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.06)
model = fpGrowth.fit(df)

In [17]:
# Display frequent itemsets.
model.freqItemsets.show(20)


+--------------------+-----+
 items| freq|
+--------------------+-----+
 [Globe Eggplant]|13016|
[Globe Eggplant, ...| 1997|
[Globe Eggplant, ...| 2376|
[Globe Eggplant, ...| 1752|
[Globe Eggplant, ...| 4687|
[Globe Eggplant, ...| 1459|
[Globe Eggplant, ...| 1717|
[Globe Eggplant, ...| 1482|
[Globe Eggplant, ...| 1909|
[Globe Eggplant, ...| 3200|
[Globe Eggplant, ...| 2647|
[Globe Eggplant, ...| 1713|
[Globe Eggplant, ...| 2699|
[Globe Eggplant, ...| 2155|
[Globe Eggplant, ...| 2373|
[Globe Eggplant, ...| 1824|
[Globe Eggplant, ...| 1964|
[Globe Eggplant, ...| 1386|
[Globe Eggplant, ...| 3861|
[Globe Eggplant, ...| 3417|
+--------------------+-----+
only showing top 20 rows

In [18]:
model.associationRules.show(5)


+--------------------+--------------------+------------------+------------------+
 antecedent| consequent| confidence| lift|
+--------------------+--------------------+------------------+------------------+
[Organic Yellow O...|[Organic Strawber...|0.4283837056504599|3.3630853397211706|
[Organic Yellow O...|[Organic Hass Avo...|0.5070083223828296| 5.722724542171714|
[Organic Yellow O...| [Organic Garlic]|0.3565484012264564| 6.658793871877157|
[Organic Yellow O...| [Organic Lemon]|0.3098992553657468| 7.201097173322273|
[Broccoli Crown, ...| [Organic Zucchini]|0.1677844588344126| 2.848106683150803|
+--------------------+--------------------+------------------+------------------+
only showing top 5 rows

In [19]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
df4 = model.transform(df).show()

+--------------------+--------+--------------------+
 items|order_id| prediction|
+--------------------+--------+--------------------+
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Raw Shrimp, Seed...| 1342|[Organic Baby Spi...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
[Cracked Wheat, S...| 1591|[Organic Raspberr...|
+--------------------+--------+--------------------+
only showing top 20 rows